In [2]:
using Pkg: @pkg_str
pkg"activate .."


Activating environment at `~/JuliaEnvs/WrapperSimplification/Project.toml`


In [3]:
using LinearAlgebra
using DataStructures
using Test

In [4]:
x = [10a + b for a in 1:3, b in 1:3]

3×3 Array{Int64,2}:
 11  12  13
 21  22  23
 31  32  33

In [22]:
# this is not a const, on purpose, we mutate it
# but we always pass it in as an default argument to the function that uses it
# thus causing that function to respecialize when it changes
ALL_RULES = ()  


trule1(::Any) = nothing
function trule1(x::Transpose{V, Transpose{V, T}}) where {V, T}
    #(score = 2, type=T, thunk=()::T->parent(parent(x)))
    (state=parent(parent(x)), score=2)
end
ALL_RULES = (ALL_RULES..., trule1)

trule2(::Any) = nothing
function trule2(x::Transpose{V,Diagonal{V,T}}) where {V,T}
    (state=parent(x), score=1)
end
ALL_RULES = (ALL_RULES..., trule2)

(trule1, trule2)

In [23]:
function simplify(x, allrules=ALL_RULES; max_steps=10)
    partial_solutions = PriorityQueue{Any, Int}(Base.Order.Reverse)
    function pend!(x, score)
        state = (typeof(x), x)
        if !haskey(partial_solutions, state) 
            enqueue!(partial_solutions, state=>score)
        end
        return nothing
    end
    function next()
        (type, val), score = peek(partial_solutions)
        return val, score
    end
    
    pend!(x, 0)
    for _ in 1:max_steps
        val, score = next()
        for rule in allrules
            res = rule(val)
            res === nothing && continue
            new_val, score_change = res
            new_score = score + score_change
            pend!(new_val, new_score)
        end
    end
    return partial_solutions
end

simplify (generic function with 2 methods)

In [24]:
simplify(x)

PriorityQueue{Any,Int64,Base.Order.ReverseOrdering{Base.Order.ForwardOrdering}} with 1 entry:
  (Array{Int64,2}, [11 12 13; 21 22 23; 31 32 33]) => 0

In [25]:
ttx = Transpose(Transpose(x))
simplify(ttx)

PriorityQueue{Any,Int64,Base.Order.ReverseOrdering{Base.Order.ForwardOrdering}} with 2 entries:
  (Array{Int64,2}, [11 12 13; 21 22 23; 31 32 33])                          => 2
  (Transpose{Int64,Transpose{Int64,Array{Int64,2}}}, [11 12 13; 21 22 23; … => 0

In [26]:
simplify(Transpose(Diagonal(x)))

PriorityQueue{Any,Int64,Base.Order.ReverseOrdering{Base.Order.ForwardOrdering}} with 2 entries:
  (Diagonal{Int64,Array{Int64,1}}, [11 0 0; 0 22 0; 0 0 33])                => 1
  (Transpose{Int64,Diagonal{Int64,Array{Int64,1}}}, [11 0 0; 0 22 0; 0 0 3… => 0

In [28]:
simplify(Transpose(Transpose(Diagonal(x))))

PriorityQueue{Any,Int64,Base.Order.ReverseOrdering{Base.Order.ForwardOrdering}} with 2 entries:
  (Diagonal{Int64,Array{Int64,1}}, [11 0 0; 0 22 0; 0 0 33])                => 2
  (Transpose{Int64,Transpose{Int64,Diagonal{Int64,Array{Int64,1}}}}, [11 0… => 0

In [29]:
Symmetric(Transpose(x))

3×3 Symmetric{Int64,Transpose{Int64,Array{Int64,2}}}:
 11  21  31
 21  22  32
 31  32  33

In [31]:
Transpose(Symmetric(x))

3×3 Transpose{Int64,Symmetric{Int64,Array{Int64,2}}}:
 11  12  13
 12  22  23
 13  23  33

In [32]:
Transpose(Symmetric(Transpose(x)))

3×3 Transpose{Int64,Symmetric{Int64,Transpose{Int64,Array{Int64,2}}}}:
 11  21  31
 21  22  32
 31  32  33

3×3 Symmetric{Int64,Array{Int64,2}}:
 11  12  13
 12  22  23
 13  23  33